# Classic SQL Agent (ReAct Architecture)

This notebook demonstrates the original LangChain SQL Agent using:

- OpenAI GPT model
- `create_sql_agent`
- ReAct reasoning pattern
- Automatic SQL tool wiring

⚠ This approach relies on text-based parsing (ReAct).

In [3]:
# Cell 2 — Install Dependencies
!pip install -r requirements.txt

In [4]:
# Cell 3 — Import Libraries

import os
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits.sql.base import create_sql_agent

In [5]:
# Cell 4 — Set API Key Securely

import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API Key: ")

In [6]:
# Cell 5 — Initialize Deterministic Model

llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,  # Deterministic output
    max_tokens=300
)

In [7]:
# Cell 6 — Connect SQLite Database
db = SQLDatabase.from_uri("sqlite:///Chinook.db")

print("Available Tables:", db.get_usable_table_names())

Available Tables: ['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


In [8]:
# Cell 7 — Create SQL Agent (ReAct)
agent = create_sql_agent(
    llm=llm,
    db=db,
    verbose=True  # Shows internal reasoning steps
)

Important:

This internally bundles:

sql_db_query

sql_db_schema

sql_db_list_tables

sql_db_query_checker

And runs a ReAct reasoning loop.

In [9]:
# Cell 8 — Ask Question
result = agent.invoke(
    {"input": "How many tracks are there in the database?"}
)

print("Final Answer:")
print(result["output"])



> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackThe "Track" table seems to be the most relevant for this question. I should query the schema of the "Track" table to see the structure of the data.
Action: sql_db_schema
Action Input: Track
CREATE TABLE "Track" (
	"TrackId" INTEGER NOT NULL, 
	"Name" NVARCHAR(200) NOT NULL, 
	"AlbumId" INTEGER, 
	"MediaTypeId" INTEGER NOT NULL, 
	"GenreId" INTEGER, 
	"Composer" NVARCHAR(220), 
	"Milliseconds" INTEGER NOT NULL, 
	"Bytes" INTEGER, 
	"UnitPrice" NUMERIC(10, 2) NOT NULL, 
	PRIMARY KEY ("TrackId"), 
	FOREIGN KEY("MediaTypeId") REFERENCES "MediaType" ("MediaTypeId"), 
	FOREIGN KEY("GenreId") REFERENCES "Genre" ("GenreId"), 
	FOREIGN KEY("AlbumId") REFERENCES "Album" ("AlbumId")
)

/*
3 rows from Track table:
TrackId	Name	AlbumId	MediaTypeId	GenreId	Composer	Milliseconds	Bytes	UnitPrice
1	For Those About 

# Cell 9 — Architecture Explanation

## What Happened Internally?

The agent followed the ReAct pattern:

1. Thought: Decide which tool to use
2. Action: sql_db_list_tables
3. Observation: Tables returned
4. Action: sql_db_query
5. Observation: Query result
6. Final Answer: Natural language response

This is text-based reasoning and depends heavily on GPT's formatting consistency.